In [2]:
import torch
from torch.utils.data import Dataset, DataLoader 

import matplotlib
import numpy as np

import os

In [3]:
"""
Arguments
data_dir: Absolute path to the dataset directory.
states_filename: Name of states dataset file.
actions_filename: Name of the actions dataset file.
s_transform: Transformation for states.
a_transform: Transformation for actions.

---------------------------------------------------------------------------------
what does it contain ?
states is a numpy array - (num of data points, trajectory_length, 2, 65, 65)
actions is a numpy array - (num of data_points, trajectory_length, 2)
transforms should be image transformations

sample implementation can be seen at : ./src/dataset/data_set_loader.ipynb

TODO: check if agent and environment needs the same transformation or different.
"""
class TrajectoryDataset(Dataset):
    def __init__(self, data_dir, 
                 states_filename, 
                 actions_filename, 
                 s_transform=None, 
                 a_transform=None):
        self.states = np.load(f"{data_dir}/{states_filename}", mmap_mode="r")
        self.actions = np.load(f"{data_dir}/{actions_filename}")
        self.state_transform = s_transform
        self.action_transform = a_transform
    
    def __len__(self):
        return self.states.shape[0]
    
    def __getitem__(self, index):
        state = self.states[index]
        action = self.actions[index]
        
        if self.state_transform:
            for i in range(state.shape[0]):
                state[i] = self.state_transform(state[i])
        
        if self.action_transform:
            for i in range(action[i].shape[0]):
                action[i] = self.action_transform(action[i])
        
        return state, action

In [4]:
traj_dataset = TrajectoryDataset(
    data_dir = "./",
    states_filename = "states.npy",
    actions_filename = "actions.npy",
    s_transform = None,
    a_transform = None
)

In [7]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataloader = DataLoader(traj_dataset, batch_size=2, shuffle=True)
print(len(dataloader))
flag = False
for batch in dataloader:
    s, a = batch
    if(not flag):
        print(s.shape)
        print(a.shape)
    
    s = s.to(device)
    a = a.to(device)

    if(not flag):
        print(f"a device: {a.device}")
        print(f"s device: {s.device}")

    flag = True

7500
torch.Size([2, 17, 2, 65, 65])
torch.Size([2, 16, 2])
a device: cuda:0
s device: cuda:0
